This notebook will explore building models with away and home teams trained in same model. Home and away have not been seperated yet.

## 1. Import Packages and Dataset 

In [1]:
import pandas as pd
import numpy as np
import sklearn
import torch.nn as nn
import sklearn.model_selection
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
pd.set_option('display.max_columns', None)

In [111]:
def add_target(team):
    team['target'] = team['won'].shift(-1)
    return team

def winrate(team):
    total = team['Wins'] + team['Losses']
    total_opp = team['Wins_opp'] + team['Losses_opp']
    team['winrate'] = team['Wins'] / total
    team['winrate_opp'] = team['Wins_opp'] / total_opp
    return team

def differential(team):
    team['differential'] = team['Total'] - team['Total_opp']
    return team

def find_team_exp_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=5, adjust=False).mean()
    return rolling

def find_team_exp_average_9(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=9, adjust=False).mean()
    return rolling

def find_team_exp_average_12(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=12, adjust=False).mean()
    return rolling

def find_team_average_15(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(15).mean()
    return rolling

def find_team_average_10(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(10).mean()
    return rolling

def find_team_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(5).mean()
    return rolling

def find_team_average_3(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(3).mean()
    return rolling

def rolling(data):
    df_rolling_3 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_3 = df_rolling_3.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_3)
    df_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_5 = df_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_5)
    df_rolling_10 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_10 = df_rolling_10.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_10)
    df_rolling_15 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_15 = df_rolling_15.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_15)
    df_exp_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_5 = df_exp_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_5)
    df_exp_rolling_9 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_9 = df_exp_rolling_9.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_9)
    df_exp_rolling_12 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_12 = df_exp_rolling_12.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_12, include_groups=False)
    exp_rolling_columns_5 = [f"{col}_exp_5" for col in df_exp_rolling_5.columns]
    exp_rolling_columns_9 = [f"{col}_exp_9" for col in df_exp_rolling_9.columns]
    exp_rolling_columns_12 = [f"{col}_exp_12" for col in df_exp_rolling_12.columns]
    rolling_columns_15 = [f"{col}_15" for col in df_rolling_15.columns]
    rolling_columns_10 = [f"{col}_10" for col in df_rolling_10.columns]
    rolling_columns_5 = [f"{col}_5" for col in df_rolling_5.columns]
    rolling_columns_3 = [f"{col}_3" for col in df_rolling_3.columns]
    df_exp_rolling_12.columns = exp_rolling_columns_12
    df_exp_rolling_9.columns = exp_rolling_columns_9
    df_exp_rolling_5.columns = exp_rolling_columns_5
    df_rolling_15.columns = rolling_columns_15
    df_rolling_10.columns = rolling_columns_10
    df_rolling_5.columns = rolling_columns_5
    df_rolling_3.columns = rolling_columns_3
    df = pd.concat([data, df_rolling_3, df_rolling_5, df_rolling_10, df_rolling_15,df_exp_rolling_5,df_exp_rolling_9, df_exp_rolling_12], axis=1)
    return df

def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))

def date_change(datetime_str):
    # Parse the datetime string into a datetime object
    datetime_obj = datetime.strptime(datetime_str, '%m/%d/%Y')

    # Format the datetime object into a new string structure
    new_datetime_str = datetime_obj.strftime('%Y-%m-%d')

    return new_datetime_str

def haircut(df, date):
    df[date] = df[date].str[:10]
    return df

def convert_date_format(df):
    # Create a boolean mask to identify values in the "m/d/y" format
    mask = df['Date'].str.contains(r'\d{1,2}/\d{1,2}/\d{2}')
    
    # Apply the conversion only to values that match the mask
    df.loc[mask, 'Date'] = nba.loc[mask, 'Date'].apply(date_change)
    
    return df

In [123]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/raw_data/NBA_2018_2024.csv"
df = pd.read_csv(folder_path, index_col=0)

folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/nba_api/data/teams_stats/processed_cumulative_season_stats_2019_2024.csv"
nba = pd.read_csv(folder_path, index_col=0)

### 2. Data Wrangling and Preprocessing for Modeling

In [128]:
# nba dataframe does not include the 2018 season
df = df[~df['season'].isin([2018])]
df = df.reset_index(drop=True)
df = haircut(df, 'date')

# rename nba columns to match df
nba = haircut(nba, 'Date')
nba = convert_date_format(nba)
nba.rename(columns={'Date': 'date_next', 'Teams':'Teams_x'}, inplace=True)

# construct winrate for team
df = winrate(df)
# construct differential points
df = differential(df)
# construct target
df = df.groupby("Teams", group_keys=False).apply(add_target)

# games yet to play are 2
df.loc[pd.isnull(df['target']), 'target'] = 2
# convert win/loss to 1/0
df['target'] = df['target'].astype(int)

# remove metadata and target
removed = ['target', 'date', 'Teams_opp', 'Teams',
           'season','won', 'Wins', 'Losses', 
           'Wins_opp', 'Losses_opp']
valid_columns = df.columns[~df.columns.isin(removed)]

# scale the data
scaler = MinMaxScaler()
df[valid_columns] = scaler.fit_transform(df[valid_columns])

# construct rolling features to df
df = rolling(df).copy()
df = df.dropna()

# construct current game metadata
df['home_next'] = add_col(df, 'home')
df['team_next_opp'] = add_col(df, 'Teams_opp')
df['date_next'] = add_col(df, 'date')
df = df.copy()

# merge stats from opposing teams
full = df.merge(df,
               left_on=['Teams', 'date_next'],
               right_on = ['team_next_opp', 'date_next'])

# merge stats from nba dataframe
complete = pd.merge(full, nba, on=['Teams_x', 'date_next'], how='left')
complete = complete.dropna()

# remove metadata and useless data
disregard = list(complete.columns[complete.dtypes == 'object']) 
disregard = disregard + ["home_opp_5_x","target_x","target_y", 
                         "Wins_x", "Losses_x", "Wins_opp_x", 
                         "Losses_opp_x", "season_x" , "won_x" , 
                         "home_5_x" ,"home_10_x" ,"season_5_x", 
                         "season_10_x" , "Wins_y" , "Losses_y" , 
                         "Wins_opp_y" , "Losses_opp_y" , "season_y" , 
                         "won_y" ,"home_5_y", "home_10_y","season_5_y", 
                         "season_10_y","home_opp_5_y", "home_opp_10_y"]
regard = complete.columns[~complete.columns.isin(disregard)]

# remove metadata and useless data
disregard_full = list(full.columns[full.dtypes == 'object']) 
disregard_full = disregard_full + ["home_opp_5_x","target_x","target_y", 
                         "Wins_x", "Losses_x", "Wins_opp_x", 
                         "Losses_opp_x", "season_x" , "won_x" , 
                         "home_5_x" ,"home_10_x" ,"season_5_x", 
                         "season_10_x" , "Wins_y" , "Losses_y" , 
                         "Wins_opp_y" , "Losses_opp_y" , "season_y" , 
                         "won_y" ,"home_5_y", "home_10_y","season_5_y", 
                         "season_10_y","home_opp_5_y", "home_opp_10_y"]
regard_full = full.columns[~full.columns.isin(disregard_full)]

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_11906/320544133.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Teams", group_keys=False).apply(add_target)
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_11906/2596421232.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_rolling_3 = df_rolling_3.groupby(['Teams', 'season'], group_keys = False).apply(find_team

In [134]:
# save processed dataframe
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/processed_data/'
file = 'processed_2019_2024.csv'
file_name = save_path + file
complete.to_csv(file_name)

## 3. Modeling

### selecting features with rankings

In [135]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

# can also try direction backward
sfs = SequentialFeatureSelector(rr, n_features_to_select = 70, direction='forward', cv=split)

In [136]:
sfs.fit(complete[regard], complete['target_x'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=70)

In [137]:
predictors = list(regard[sfs.get_support()])

In [138]:
predictors

['efg%_x',
 'usg%_x',
 '+/-maxes_x',
 'eFG%_x',
 'ts%_opp_x',
 'usg%_opp_x',
 '3pmaxes_opp_x',
 'tov%maxes_opp_x',
 'winrate_x',
 'blk%_3_x',
 'usg%_3_x',
 'ptsmaxes_3_x',
 'usg%_opp_3_x',
 'winrate_3_x',
 'usg%_5_x',
 'usg%_opp_5_x',
 'ft%maxes_opp_5_x',
 'ft_10_x',
 'usg%_10_x',
 'usg%_opp_10_x',
 '3pmaxes_opp_10_x',
 'drb_15_x',
 'efg%_15_x',
 'usg%_15_x',
 'eFG%_15_x',
 'usg%_opp_15_x',
 'usg%_exp_5_x',
 'usg%_opp_exp_5_x',
 'usg%_exp_9_x',
 'usg%_opp_exp_9_x',
 'usg%_exp_12_x',
 'usg%_opp_exp_12_x',
 'ft%maxes_opp_exp_12_x',
 '3p%_y',
 'usg%_y',
 'usg%_opp_y',
 'usg%_3_y',
 'fgamaxes_3_y',
 'ft%_opp_3_y',
 'usg%_opp_3_y',
 'winrate_3_y',
 'usg%_5_y',
 'usg%_opp_5_y',
 'usg%_10_y',
 'usg%_opp_10_y',
 'tovmaxes_opp_10_y',
 'usg%_15_y',
 'usg%_opp_15_y',
 'usg%_exp_5_y',
 'usg%_opp_exp_5_y',
 'usg%_exp_9_y',
 'usg%_opp_exp_9_y',
 'usg%_exp_12_y',
 'ft%_opp_exp_12_y',
 'usg%_opp_exp_12_y',
 'W_PCT_base',
 'L_RANK_base',
 'W_PCT_RANK_base',
 'E_NET_RATING_advanced',
 'L_RANK_advanced',

In [153]:
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import accuracy_score,f1_score
import pickle 

def backtest(data, model, predictors, true):
    X = data[predictors]
    y = data['target_x']
    
    test_size = len(y) // 4
    
    best_accuracy = 0
    best_f1 = 0
    best_model = None
    fold = 0
    
    tscv = TimeSeriesSplit(n_splits=3, test_size=test_size)
    accuracy_scores = []
    
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        
        accuracy = accuracy_score(y_test, predictions)
        f1 = f1_score(y_test, predictions)
        accuracy_scores.append(accuracy)
    
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_f1 = f1
            best_model = model
            fold = len(accuracy_scores)

        print(f'Accuracy for fold {len(accuracy_scores)}: {accuracy}')
        print(f'F1 for fold {len(accuracy_scores)}: {f1}')
        print("\n")

    if (True):
        count = len(predictors)
        accuracy = round(best_accuracy, 3) * 1000
        save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/weights/'
        file_path = save_path + f'ridge_classifier_{count}_predictors_{accuracy}%_2019_2024.pkl'
        with open(file_path, 'wb') as f:
            pickle.dump(best_model, f)
        
        save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/factors/'
        file_path = save_path + f'predictors_ridge_classifier_{count}_predictors_{accuracy}%_2019_2024.txt'
        with open(file_path, 'w') as f:
            for predictor in predictors:
                f.write(f'{predictor},')
    
        print('Saved the best model into directory')

In [154]:
backtest(complete, rr, predictors, True)

Accuracy for fold 1: 0.8004484304932735
F1 for fold 1: 0.8019287833827893


Accuracy for fold 2: 0.8501494768310912
F1 for fold 2: 0.8506517690875233


Accuracy for fold 3: 0.8677130044843049
F1 for fold 3: 0.8670172802404208


Saved the best model into directory


### selecting features without rankings

In [5]:
full

mp_x      fg_x     fga_x     fg%_x      3p_x     3pa_x     3p%_x  \
0      0.00  0.727273  0.516667  0.705742  0.407407  0.350000  0.452781   
1      0.00  0.250000  0.483333  0.188995  0.259259  0.533333  0.187581   
2      0.00  0.363636  0.400000  0.373206  0.037037  0.133333  0.126779   
3      0.00  0.454545  0.516667  0.399522  0.407407  0.383333  0.420440   
4      0.00  0.363636  0.416667  0.361244  0.222222  0.316667  0.267788   
...     ...       ...       ...       ...       ...       ...       ...   
10711  0.00  0.318182  0.433333  0.296651  0.370370  0.383333  0.381630   
10712  0.00  0.409091  0.350000  0.466507  0.444444  0.366667  0.477361   
10713  0.00  0.431818  0.416667  0.444976  0.222222  0.300000  0.280724   
10714  0.25  0.477273  0.516667  0.425837  0.333333  0.416667  0.316947   
10715  0.00  0.522727  0.466667  0.514354  0.518519  0.516667  0.415265   

           ft_x     fta_x     ft%_x     orb_x     drb_x     trb_x     ast_x  \
0      0.404762  0.431373  0.674355  0.379310  0.659091  0.653846  0.619048   
1      0.428571  0.470588  0.648575  0.517241  0.272727  0.403846  0.285714   
2      0.595238  0.607843  0.720488  0.413793  0.431818  0.480769  0.238095   
3      0.166667  0.196078  0.582090  0.137931  0.363636  0.269231  0.333333   
4      0.285714  0.333333  0.592944  0.379310  0.477273  0.500000  0.190476   
...         ...       ...       ...       ...       ...       ...       ...   
10711  0.166667  0.215686  0.515604  0.379310  0.500000  0.519231  0.357143   
10712  0.238095  0.215686  0.805970  0.482759  0.386364  0.480769  0.404762   
10713  0.404762  0.431373  0.674355  0.448276  0.318182  0.403846  0.309524   
10714  0.571429  0.568627  0.746269  0.517241  0.636364  0.711538  0.523810   
10715  0.190476  0.176471  0.773406  0.413793  0.159091  0.250000  0.452381   

          stl_x  blk_x     tov_x     pf_x     pts_x     ts%_x    efg%_x  \
0      0.545455   0.30  0.482759  0.56250  0.648148  0.665929  0.672065   
1      0.318182   0.25  0.620690  0.50000  0.231481  0.203540  0.194332   
2      0.227273   0.20  0.413793  0.53125  0.333333  0.340708  0.287449   
3      0.363636   0.15  0.379310  0.28125  0.333333  0.365044  0.412955   
4      0.272727   0.05  0.586207  0.25000  0.259259  0.309735  0.334008   
...         ...    ...       ...      ...       ...       ...       ...   
10711  0.363636   0.15  0.482759  0.21875  0.212963  0.269912  0.323887   
10712  0.272727   0.15  0.551724  0.21875  0.333333  0.488938  0.500000   
10713  0.318182   0.25  0.413793  0.56250  0.361111  0.407080  0.402834   
10714  0.318182   0.50  0.551724  0.59375  0.490741  0.451327  0.412955   
10715  0.409091   0.45  0.275862  0.28125  0.425926  0.522124  0.546559   

         3par_x     ftr_x    orb%_x    drb%_x    trb%_x    ast%_x    stl%_x  \
0      0.379074  0.330028  0.505376  0.551971  0.682320  0.573190  0.528037   
1      0.596913  0.368272  0.448029  0.494624  0.317680  0.546995  0.308411   
2      0.168096  0.507082  0.467742  0.514337  0.500000  0.332820  0.233645   
3      0.415094  0.148725  0.148746  0.541219  0.212707  0.411402  0.355140   
4      0.379074  0.274788  0.437276  0.648746  0.574586  0.249615  0.280374   
...         ...       ...       ...       ...       ...       ...       ...   
10711  0.449400  0.175637  0.394265  0.720430  0.544199  0.596302  0.392523   
10712  0.466552  0.189802  0.677419  0.521505  0.701657  0.571649  0.313084   
10713  0.358491  0.355524  0.582437  0.458781  0.549724  0.392912  0.350467   
10714  0.451115  0.434844  0.548387  0.544803  0.627072  0.684129  0.308411   
10715  0.586621  0.140227  0.478495  0.322581  0.298343  0.530046  0.467290   

         blk%_x    tov%_x  usg%_x    ortg_x    drtg_x  fgmaxes_x  fgamaxes_x  \
0      0.297030  0.429104     0.0  0.677313  0.210352   0.272727    0.129032   
1      0.195545  0.541045     0.0  0.210352  0.677313   0.090909    0.161290   
2      0.173267  0.384328     0.0  0.386564  0.507709   0.272727   

In [6]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

# can also try direction backward
sfs_full = SequentialFeatureSelector(rr, n_features_to_select = 50, direction='forward', cv=split)

In [7]:
sfs_full.fit(full[regard_full], full['target_x'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=50)

In [9]:
predictors_full = list(regard_full[sfs_full.get_support()])

In [10]:
predictors_full

['usg%_x',
 'usg%_opp_x',
 'usg%_3_x',
 'usg%_opp_3_x',
 'stl%maxes_opp_3_x',
 'usg%_5_x',
 'ft%maxes_5_x',
 'orb_opp_5_x',
 'usg%_opp_5_x',
 'stl%maxes_opp_5_x',
 'winrate_5_x',
 'usg%_10_x',
 'ftr_opp_10_x',
 'usg%_opp_10_x',
 'orb%maxes_opp_10_x',
 'usg%_15_x',
 'ft%_opp_15_x',
 'usg%_opp_15_x',
 'usg%_exp_5_x',
 'usg%maxes_exp_5_x',
 'usg%_opp_exp_5_x',
 'usg%_exp_9_x',
 'ft%maxes_exp_9_x',
 'usg%_opp_exp_9_x',
 'drb%_exp_12_x',
 'usg%_exp_12_x',
 'usg%_opp_exp_12_x',
 'usg%_y',
 'stlmaxes_y',
 'usg%_opp_y',
 'winrate_y',
 'usg%_3_y',
 'usg%_opp_3_y',
 'usg%_5_y',
 'usg%_opp_5_y',
 'usg%_10_y',
 'usg%_opp_10_y',
 'trb_15_y',
 'usg%_15_y',
 'usg%_opp_15_y',
 'usg%_exp_5_y',
 'pfmaxes_exp_5_y',
 'usg%_opp_exp_5_y',
 'usg%_exp_9_y',
 'usg%_opp_exp_9_y',
 'orb%maxes_opp_exp_9_y',
 'trb%maxes_opp_exp_9_y',
 'usg%_exp_12_y',
 'usg%_opp_exp_12_y',
 'trb%maxes_opp_exp_12_y']

In [19]:
final_full = backtest(full, rr, predictors_full, True)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -4.52642427e-02  0.00000000e+00 -1.56891833e-01  3.46364438e-01
   0.00000000e+00 -5.30906549e-02  1.29801220e+00  0.00000000e+00
   6.58634722e-02  0.00000000e+00 -1.06227413e+00  0.00000000e+00
  -1.74399716e-01  0.00000000e+00  0.00000000e+00  7.22861362e-01
   0.00000000e+00  0.00000000e+00  2.08353502e-01  0.00000000e+00
   7.31613827e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.21163089e-02  0.00000000e+00 -1.41112423e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -1.26591360e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  8.87047510e-05  0.00000000e+00  0.00000000e+00
   0.00000000e+00  9.64050584e-01 -3.41324731e-01  0.00000000e+00
   0.00000000e+00  3.75649957e-01]]


In [14]:
accuracy_full = accuracy_score(final_full['Actual'], final_full['Predictions'])
f1_full = f1_score(final_full['Actual'], final_full['Predictions'])

In [15]:
print("The accuracy is: ", accuracy_full)
print("The f1 is: ", f1_full)

The accuracy is:  0.6557835820895522
The f1 is:  0.6512287334593574


In [16]:
# save the predictors 
file_path = 'factors_ridge_classifier_70_features_no_rankigns_65%__2019-2024.txt'
with open(file_path, 'w') as f:
    for predictor in predictors_full:
        f.write(f'{predictor},')

In [ ]:
# I believe there is an error in backtest